## Configure PySpark Setup

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"


import findspark
findspark.init()


import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("App").getOrCreate()
spark

In [2]:
# check number of cores PySpark is using
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")

You are working with 1 core(s)


## Load Datasets


In [3]:
!cp /content/drive/MyDrive/Datasets.zip .
!unzip Datasets.zip

Archive:  Datasets.zip
  inflating: Datasets/.DS_Store      
  inflating: Datasets/fifa19.csv     
  inflating: Datasets/googleplaystore.csv  
  inflating: Datasets/nyc_air_bnb.csv  
  inflating: Datasets/people.json    
  inflating: Datasets/pga_tour_historical.csv  
  inflating: Datasets/rec-crime-pfa.csv  
  inflating: Datasets/Rep_vs_Dem_tweets.csv  
  inflating: Datasets/students.csv   
  inflating: Datasets/supermarket_sales.csv  
  inflating: Datasets/users1.parquet  
  inflating: Datasets/users2.parquet  
  inflating: Datasets/users3.parquet  
   creating: Datasets/uw-madison-courses/
  inflating: Datasets/uw-madison-courses/course_offerings.csv  
  inflating: Datasets/uw-madison-courses/courses.csv  
  inflating: Datasets/uw-madison-courses/database.sqlite3  
  inflating: Datasets/uw-madison-courses/grade_distributions.csv  
  inflating: Datasets/uw-madison-courses/instructors.csv  
  inflating: Datasets/uw-madison-courses/rooms.csv  
  inflating: Datasets/uw-madison-courses/s

# Load Libraries

In [26]:
from pyspark.sql.functions  import *
from  pyspark.sql.types  import *

## Reading Data into PySpark

In [7]:
path='Datasets/'
data = spark.read.csv(path+'nyc_air_bnb.csv',inferSchema=True,header=True)

In [8]:
data.limit(3).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365


In [9]:
print(data.printSchema())

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)

None


In [27]:
# count missing data for each column
data.select([count(when(isnan(c) | data[c].isNull(), c)).alias(c) for c in data.columns]).show()

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
| id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  0|  32|    185|      206|                185|          185|     185|      185|      185|  185|           185|              205|      10234|            10215|                           187|             342|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+----

In [28]:
# change data type

df = data.withColumn("price", data["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", data["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", data["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", data["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", data["calculated_host_listings_count"].cast(IntegerType()))

print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


In [32]:
# groupby and count 
df.groupBy("neighbourhood_group").count().show(5)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
|            Midtown|    4|
|    Jackson Heights|    2|
+-------------------+-----+
only showing top 5 rows



In [34]:
# groupby and count and sorting
df.groupBy("neighbourhood_group").count().orderBy('count').show(5)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|        Little Neck|    1|
|       Clinton Hill|    1|
|          Briarwood|    1|
|              Nadia|    1|
|         Douglaston|    1|
+-------------------+-----+
only showing top 5 rows



In [35]:
# groupby and count and sorting
df.groupBy("neighbourhood_group").count().orderBy(col('count').desc()).show(5)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|          Manhattan|21594|
|           Brooklyn|20055|
|             Queens| 5630|
|              Bronx| 1080|
|      Staten Island|  370|
+-------------------+-----+
only showing top 5 rows



In [39]:
# groupby and aggregation
df.groupBy("neighbourhood").mean('price').show(5)


+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



In [38]:
df.groupBy("neighbourhood").agg({'price':'mean'}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



In [40]:
# using pyspark sql functions in aggregate
df.groupBy("neighbourhood").agg(min(df['price']).alias("Min Price"),max(df['price']).alias("Max Price")).show(5)

+-------------+---------+---------+
|neighbourhood|Min Price|Max Price|
+-------------+---------+---------+
|       Corona|       23|      359|
| Richmondtown|       78|       78|
| Prince's Bay|       85|     1250|
|  Westerleigh|       40|      103|
|   Mill Basin|       85|      299|
+-------------+---------+---------+
only showing top 5 rows



In [42]:
df.groupBy("neighbourhood").agg(min(df['price']).alias("Min Price"),max(df['price']).alias("Max Price")).orderBy(col('Max Price').desc()).show(5)

+---------------+---------+---------+
|  neighbourhood|Min Price|Max Price|
+---------------+---------+---------+
|        Astoria|       25|    10000|
|Upper West Side|       10|    10000|
|     Greenpoint|        0|    10000|
|Lower East Side|       29|     9999|
|    East Harlem|       30|     9999|
+---------------+---------+---------+
only showing top 5 rows



In [43]:
summary = df.summary("count", "min", "25%", "75%", "max")
summary.toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,count,49079,49047,48894,48873,48894,48894,48894,48894,48894,48887,48891,48738,38845,38858,48891,48737
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-74,0,0,-73.94134,0,0,0
2,25%,9470677.0,2.4544724E7,7798275.0,475.0,1.94716858E8,40.68771,40.68982,-73.9831,56.0,69,1,1,0.76,0,1,0
3,75%,2.9150743E7,1.74786681E8,1.07434423E8,3.3499491E7,1.97400421E8,40.78304,40.76299,-73.93638,145.0,175,5,23,3.24,2,2,226
4,max,"獨一無二的紐約閣樓""","ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",呈刚,현선,Woodside,Woodside,West Village,Shared room,Shared room,10000,1250,629,9.66,58,365,365


In [44]:
limit_summary = df.select("price","minimum_nights","number_of_reviews").summary("count","min","max")
limit_summary.toPandas()

,summary,price,minimum_nights,number_of_reviews
0,count,48887,48891,48738
1,min,-74,0,0
2,max,10000,1250,629


In [47]:
# just aggregations
df.agg(min(df.price).alias("Min Price"),max(df.price).alias("Max Price")).show()

+---------+---------+
|Min Price|Max Price|
+---------+---------+
|      -74|    10000|
+---------+---------+



In [48]:
df.select(countDistinct("neighbourhood_group").alias('CountD'),avg('price'),stddev("price")).show()

+------+------------------+------------------+
|CountD|        avg(price)|stddev_samp(price)|
+------+------------------+------------------+
|    77|152.22298361527604| 238.5414668883948|
+------+------------------+------------------+



In [49]:
df.agg({'number_of_reviews':'max'}).withColumnRenamed("max(number_of_reviews)", "Max Reviews").show()

+-----------+
|Max Reviews|
+-----------+
|        629|
+-----------+



In [50]:
# pyspark pivot
df.groupBy("room_type").pivot("neighbourhood_group").count().limit(5).toPandas()

,room_type,null,194716858,197400421,Arverne,Astoria,Bath Beach,Bedford-Stuyvesant,Borough Park,Briarwood,Bronx,Brooklyn,Brooklyn Heights,Bushwick,Cambria Heights,Canarsie,Carmen,Chelsea,Clinton Hill,Concourse Village,Crown Heights,Cypress Hills,D,Ditmars Steinway,Douglaston,East Elmhurst,East Flatbush,East Harlem,East New York,East Village,Edgemere,Elmhurst,Eltingville,Evelyn,Flatbush,Flushing,Fort Greene,Gramercy,Greenpoint,Greenwich Village,Harlem,Hell's Kitchen,Inwood,Jackson Heights,Krista,Little Neck,Long Island City,Longwood,Manhattan,Maspeth,Midtown,Midwood,Morningside Heights,Mott Haven,Murray Hill,Nadia,NoHo,Pelham Gardens,Prospect Heights,Prospect-Lefferts Gardens,Queens,Queens Village,Randall Manor,Red Hook,Seth,SoHo,South Ozone Park,Springfield Gardens,Staten Island,Stuyvesant Town,Throgs Neck,Upper East Side,Upper West Side,Washington Heights,West Village,William Hakan,Williamsburg,Woodhaven,Woodside
0,51,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,2.0,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
1,205,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
2,54,None,None,None,None,None,None,None,None,None,None,None,None,1.0,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
3,200,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
4,279,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0,None,None,None,None,None,None,None,None,None


In [51]:
# pyspark pivot
df.groupBy("room_type").pivot("neighbourhood_group").count().na.fill(0).limit(5).toPandas()

,room_type,null,194716858,197400421,Arverne,Astoria,Bath Beach,Bedford-Stuyvesant,Borough Park,Briarwood,Bronx,Brooklyn,Brooklyn Heights,Bushwick,Cambria Heights,Canarsie,Carmen,Chelsea,Clinton Hill,Concourse Village,Crown Heights,Cypress Hills,D,Ditmars Steinway,Douglaston,East Elmhurst,East Flatbush,East Harlem,East New York,East Village,Edgemere,Elmhurst,Eltingville,Evelyn,Flatbush,Flushing,Fort Greene,Gramercy,Greenpoint,Greenwich Village,Harlem,Hell's Kitchen,Inwood,Jackson Heights,Krista,Little Neck,Long Island City,Longwood,Manhattan,Maspeth,Midtown,Midwood,Morningside Heights,Mott Haven,Murray Hill,Nadia,NoHo,Pelham Gardens,Prospect Heights,Prospect-Lefferts Gardens,Queens,Queens Village,Randall Manor,Red Hook,Seth,SoHo,South Ozone Park,Springfield Gardens,Staten Island,Stuyvesant Town,Throgs Neck,Upper East Side,Upper West Side,Washington Heights,West Village,William Hakan,Williamsburg,Woodhaven,Woodside
0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,54,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [52]:
df.groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(10)

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|         51|  null|    null|
|        205|  null|    null|
|         54|  null|    null|
|        200|  null|    null|
|        279|  null|    null|
|        138|  null|    null|
|         69|  null|    null|
|         42|  null|    null|
|Shared room|   198|     413|
|  -73.95777|  null|    null|
+-----------+------+--------+
only showing top 10 rows



In [53]:
df.filter("room_type='Shared room'").groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).count().show(100)

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|Shared room|   198|     413|
+-----------+------+--------+



In [54]:
df.groupBy("neighbourhood").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).agg(min(df['price']).alias("Min Price"),max(df['price']).alias("Max Price")).toPandas()


,neighbourhood,Queens_Min Price,Queens_Max Price,Brooklyn_Min Price,Brooklyn_Max Price
0,Corona,23.0,359.0,NaN,NaN
1,Prince's Bay,NaN,NaN,NaN,NaN
2,Richmondtown,NaN,NaN,NaN,NaN
3,Mill Basin,NaN,NaN,85.0,299.0
4,Westerleigh,NaN,NaN,NaN,NaN
...,...,...,...,...,...
378,40.69383,NaN,NaN,NaN,NaN
379,Morningside Heights,NaN,NaN,NaN,NaN
380,Greenpoint,NaN,NaN,0.0,10000.0
381,Elmhurst,15.0,443.0,NaN,NaN
